In [7]:
from bokeh.io import output_notebook, show
from bokeh.models import CustomJS, ColumnDataSource, Slider, CDSView, IndexFilter
from bokeh.models.widgets import Select
from bokeh.layouts import column
from bokeh.plotting import figure
import pandas as pd
import numpy as np

#Set output to notebook
output_notebook()

#Read in data, set index to country, and remove missing values
df = pd.read_csv("labor_15-64.csv")
df = df.set_index('Country')
df = df.dropna(axis = 1, how = 'all')
df = df.dropna(axis = 0, how = 'all')

#Create a select using country names
select = Select(title="Country:", value=df.index.values[0], options=df.index.values.tolist())
df_filtered = df.loc[df.index.values[0]]

#Setup data sources
source = ColumnDataSource(df.T)
source_filtered = ColumnDataSource(data=dict(x=df_filtered.index.values, y=df_filtered.values))

#Setup plots
p = figure(width=800, height=400, title = "Aged 15-64 Labor Force Participation Rate")
p.line('x', 'y', source=source_filtered, color='navy', alpha=0.5)

#CallbackJS for country selection
update_plot = CustomJS(args=dict(source=source, select=select, source_filtered=source_filtered), code="""
    var data = source.get('data')
    var data_f = source_filtered.get('data');
    country = select.value
    data = data[country]
    y = data_f['y']
    for(i = 0; i < y.length; i++){
        y[i] = data[i]
    }
    source_filtered.change.emit()
""")
select.js_on_change('value', update_plot)

#Display app
show(column(select, p))


Loading BokehJS ...